# * Ex-T Revenue & Subs

## Parameter

In [16]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [17]:
''' Execute Summany Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/ 
        SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH, REMARK, METRIC_CD, METRIC_NAME
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) CCAATT
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM (
        SELECT TM_KEY_DAY
            , CASE 	WHEN METRIC_CD LIKE 'TB%S%' 
                    THEN ROW_NUMBER() OVER (PARTITION BY SUBSTR(TM_KEY_DAY,1,6), METRIC_CD, AREA_CD ORDER BY TM_KEY_DAY DESC)
                    ELSE 1 END FLAG
            , CASE WHEN METRIC_CD LIKE 'TB%S%' THEN 'SUB' ELSE 'REV' END AGG_TYPE
            , METRIC_CD, METRIC_NAME, ACTUAL, COMP_CD, VERSION, AREA_CD, AREA_DESC, AREA_TYPE, LOAD_DATE, REMARK, SRC
        FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
        WHERE REMARK IN ('salisa', 'jutar11')
        AND NOT REGEXP_LIKE(METRIC_CD, '^TCAP|CORP$')
        AND EXISTS (
            SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
            FROM (
                SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            ) TMP
            WHERE TMP_CD = AREA_CD )
        AND TM_KEY_DAY >= {v_start_date} 
    ) TMP
    WHERE FLAG = 1
    GROUP BY REMARK, METRIC_CD, METRIC_NAME, SUBSTR(TM_KEY_DAY,1,6)
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summany DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summany DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101



TDMDBPR : Connected

   -> Execute query... 2025-03-12, 10:12:00

   -> Summany DataFrame : 418 rows, 17 columns

TDMDBPR : Disconnected


In [18]:
''' Current Month Revenue '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
rev_metric_list = ['TB1R000100', 'TB2R000100', 'TB2R010100', 'TB2R020100', 'TB3R000100', 'TB4R000100']

src_t_rev_overall_df = chk_src_df.copy()
src_t_rev_overall_df = src_t_rev_overall_df.loc[src_t_rev_overall_df['TM_KEY_MTH']==curr_mth]
src_t_rev_overall_df = src_t_rev_overall_df.loc[src_t_rev_overall_df['METRIC_CD'].isin(rev_metric_list)]
src_t_rev_overall_df = src_t_rev_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_overall_df = src_t_rev_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_rev_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_rev_overall_df[col] = src_t_rev_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_rev_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1R000100,Prepay Revenue,2025-03-11 04:46:38,20250301,20250308,0,0,"713,684,435","713,679,832","713,679,832","710,593,711",0,5,1104,8832
1,TB2R000100,Postpaid Revenue,2025-03-11 04:20:06,20250302,20250306,0,"899,834,139","899,650,448","899,649,549","899,649,549",0,0,5,177,354
2,TB2R010100,Postpaid Revenue B2C,2025-03-11 04:20:53,20250302,20250306,0,"755,123,737","755,018,504","755,017,605","755,017,605","752,226,113",0,6,1105,2210
3,TB2R020100,Postpaid Revenue B2B,2025-03-11 04:20:38,20250302,20250306,0,"144,710,401","144,631,943","144,631,943","144,631,943",0,0,5,177,354
4,TB3R000100,Broadband Revenue,2025-03-11 05:17:44,20250302,20250306,0,0,"361,511,239","361,511,239","361,511,239","361,584,111",0,5,1102,2203


In [19]:
''' Average Revenue per Month '''

avg_range_start = '202401'
avg_range_end = '202502'
rev_metric_list = ['TB1R000100', 'TB2R000100', 'TB2R010100', 'TB2R020100', 'TB3R000100', 'TB4R000100']

src_t_rev_avg_mth_df = chk_src_df.copy()
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.loc[src_t_rev_avg_mth_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.loc[src_t_rev_avg_mth_df['METRIC_CD'].isin(rev_metric_list)]
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_rev_avg_mth_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_rev_avg_mth_df[col] = src_t_rev_avg_mth_df[col].apply(lambda x: format(x, ',.0f'))

src_t_rev_avg_mth_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1R000100,Prepay Revenue,2025-03-10 04:44:49,20240101,20250228,0,0,"2,300,952,636","2,331,094,199","2,331,094,199","2,321,368,786",0,5,1104,469200
1,TB2R000100,Postpaid Revenue,2025-03-10 04:20:12,20240102,20250228,0,"3,702,992,293","3,643,684,481","3,667,349,135","3,667,349,135",0,0,5,177,24603
2,TB2R010100,Postpaid Revenue B2C,2025-03-10 04:21:30,20240102,20250228,0,"3,294,112,700","3,259,758,655","3,281,928,428","3,281,929,398","3,258,167,354",0,6,1105,148232
3,TB2R020100,Postpaid Revenue B2B,2025-03-10 04:21:04,20240102,20250228,0,"408,879,593","383,925,826","385,420,708","385,420,708",0,0,5,177,24603
4,TB3R000100,Broadband Revenue,2025-03-07 05:10:02,20240102,20250228,0,0,"1,573,227,163","1,573,227,163","1,573,196,204","1,560,922,362",0,5,1102,138718
5,TB4R000100,TVS Revenue,2025-02-24 05:11:52,20240102,20250222,0,0,"319,526,647","313,831,724","318,180,582","322,352,854",0,5,1104,105949


In [20]:
''' Current Month Subscription '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
sub_metric_list = ['TB1S000500', 'TB1S000600', 'TB2S000500', 'TB2S010500', 'TB2S020400', 'TB3S000500', 'TB4S000500']

src_t_sub_overall_df = chk_src_df.copy()
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['TM_KEY_MTH']==curr_mth]
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_t_sub_overall_df = src_t_sub_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_sub_overall_df = src_t_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_sub_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_sub_overall_df[col] = src_t_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_sub_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1S000500,Prepay Usage Subs,2025-03-11 04:50:20,20250309,20250309,0,0,"17,533,705","17,533,705","17,533,705","17,471,803",0,5,1104,1104
1,TB1S000600,Prepay Active Caller Subs,2025-03-11 04:50:25,20250309,20250309,0,"14,549,847","14,362,480","14,362,480","14,362,480","14,313,994",0,6,1105,1105
2,TB2S000500,Postpaid Active Subs,2025-03-11 05:36:20,20250310,20250310,0,"10,178,615","10,178,061","10,178,048","10,178,048","10,102,989",0,6,1105,1105
3,TB2S010500,Postpaid Active Subs B2C,2025-03-11 05:36:49,20250310,20250310,0,"8,997,137","8,996,892","8,996,879","8,996,879","8,941,830",0,6,1105,1105
4,TB2S020400,Postpaid Active Subs B2B,2025-03-11 05:35:53,20250310,20250310,0,"1,181,478","1,181,169","1,181,169","1,181,169",0,0,5,177,177
5,TB3S000500,Broadband Active Subs,2025-03-11 04:55:28,20250310,20250310,0,0,"3,117,135","3,117,135","3,117,135","3,117,135",0,5,1102,1102
6,TB4S000500,TVS Active Subs,2025-03-11 04:57:34,20250310,20250310,0,0,"1,160,440","1,160,440","1,160,440","1,160,425",0,5,1104,1104


In [21]:
''' Average Subscription per Month '''

avg_range_start = '202401'
avg_range_end = '202502'
sub_metric_list = ['TB1S000500', 'TB1S000600', 'TB2S000500', 'TB2S010500', 'TB2S020400', 'TB3S000500', 'TB4S000500']

src_t_sub_overall_df = chk_src_df.copy()
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_t_sub_overall_df = src_t_sub_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_sub_overall_df = src_t_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_sub_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_sub_overall_df[col] = src_t_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_sub_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1S000500,Prepay Usage Subs,2025-03-03 04:50:27,20240131,20250228,0,0,"17,638,410","17,652,566","17,652,566","17,591,038",0,5,1104,15456
1,TB1S000600,Prepay Active Caller Subs,2025-03-07 04:50:40,20240131,20250228,0,"14,441,024","14,248,564","14,362,024","14,362,051","14,312,460",0,6,1105,15470
2,TB2S000500,Postpaid Active Subs,2025-03-10 04:31:27,20240131,20250228,0,"10,021,684","9,862,550","9,923,667","9,923,692","9,851,246",0,6,1105,15470
3,TB2S010500,Postpaid Active Subs B2C,2025-03-10 04:31:57,20240131,20250228,0,"8,891,594","8,828,202","8,883,744","8,883,768","8,212,087",0,6,1105,15470
4,TB2S020400,Postpaid Active Subs B2B,2025-03-04 02:37:03,20240131,20250228,0,"1,130,090","1,034,348","1,039,923","1,039,923",0,0,5,177,2478
5,TB3S000500,Broadband Active Subs,2025-03-01 04:55:28,20240131,20250228,0,0,"3,053,895","3,055,196","3,055,196","3,055,841",0,5,1102,15412
6,TB4S000500,TVS Active Subs,2025-03-01 04:55:38,20240131,20250228,0,0,"1,220,146","1,219,989","1,219,989","1,307,223",0,5,8408,22760


In [22]:
''' METRIC Summary '''

src_t_rev_mth_df = chk_src_df.copy()
src_t_rev_mth_df = src_t_rev_mth_df.groupby(['REMARK', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_mth_df = src_t_rev_mth_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD']).reset_index()

mod_col_list = src_t_rev_mth_df.iloc[:, 6:7].columns.tolist()
for col in mod_col_list:
    src_t_rev_mth_df[col] = src_t_rev_mth_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\n{src_t_rev_mth_df.to_string(max_cols=10)}')


     REMARK      METRIC_CD                 METRIC_NAME           LOAD_DATE  START_DAY   END_DAY                  P  CNT_AREA_TYPE  CNT_AREA_CD  ROW_CNT
0   jutar11     TB1R000100              Prepay Revenue 2025-03-11 04:46:38   20240101  20250308                  0              5         1104   478032
1   jutar11     TB1R000101          Prepay New Revenue 2025-03-11 04:43:14   20240101  20250308                  0              4          176    76208
2   jutar11     TB1R000102     Prepay Existing Revenue 2025-03-11 04:45:30   20240101  20250308                  0              4          176    76208
3   jutar11     TB1S000500           Prepay Usage Subs 2025-03-11 04:50:20   20240131  20250309                  0              5         1104    16560
4   jutar11     TB1S000600   Prepay Active Caller Subs 2025-03-11 04:50:25   20240131  20250309        216,724,185              6         1105    16575
5   jutar11     TB2R000100            Postpaid Revenue 2025-03-11 04:20:06   20240102  

In [23]:
''' METRIC Current Month '''

# v_tm_key_mth = '202502'
curr_mth = chk_src_df['TM_KEY_MTH'].max()
src_t_rev_mth_df_cols = ['REMARK', 'METRIC_CD', 'METRIC_NAME', 'LOAD_DATE', 'TM_KEY_MTH', 'START_DAY', 'END_DAY', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT']
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_mth: {curr_mth}')

src_t_rev_mth_df = chk_src_df[src_t_rev_mth_df_cols].copy()
src_t_rev_mth_df = src_t_rev_mth_df.loc[src_t_rev_mth_df['TM_KEY_MTH']==curr_mth]
src_t_rev_mth_df = src_t_rev_mth_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_MTH']).reset_index(drop=True)

mod_col_list = src_t_rev_mth_df.iloc[:, 7:14].columns.tolist()
for col in mod_col_list:
    src_t_rev_mth_df[col] = src_t_rev_mth_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\n{src_t_rev_mth_df.to_string(max_cols=17)}')


Parameter input...

   -> v_tm_key_mth: 202503

     REMARK      METRIC_CD                 METRIC_NAME           LOAD_DATE TM_KEY_MTH  START_DAY   END_DAY  C            P            G            H           HH         CCAA CCAATT
0   jutar11     TB1R000100              Prepay Revenue 2025-03-11 04:46:38     202503   20250301  20250308  0            0  713,684,435  713,679,832  713,679,832  710,593,711      0
1   jutar11     TB1R000101          Prepay New Revenue 2025-03-11 04:43:14     202503   20250301  20250308  0            0   82,198,974   82,198,974   82,198,974            0      0
2   jutar11     TB1R000102     Prepay Existing Revenue 2025-03-11 04:45:30     202503   20250301  20250308  0            0  631,485,462  631,480,858  631,480,858            0      0
3   jutar11     TB1S000500           Prepay Usage Subs 2025-03-11 04:50:20     202503   20250309  20250309  0            0   17,533,705   17,533,705   17,533,705   17,471,803      0
4   jutar11     TB1S000600   Prepay Activ

## Actual : Transaction
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [24]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+ PARALLEL(8)*/ 
         REMARK, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) CCAATT
    FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
    --WHERE METRIC_CD = 'TB3R000100' --TOL Revenue
    WHERE METRIC_CD IN (
        'TB1R000100' --'Prepay Revenue
        , 'TB2R010100' --Postpaid Revenue B2C
        , 'TB3R000100' --TOL Revenue
        , 'TB4R000100' --TVS Revenue
         )
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY REMARK, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Daily DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20250101



TDMDBPR : Connected

   -> Execute query... 2025-03-12, 10:12:27

   -> Daily DataFrame : 125 rows, 12 columns

TDMDBPR : Disconnected


In [25]:
''' METRIC by Day '''

v_tm_key_day = 20250101
# src_t_rev_day_df = ['SRC', 'REMARK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT', 'LOAD_DATE']
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

# src_t_rev_day_df = chk_src_df[src_t_rev_day_df_cols].copy()
src_t_rev_day_df = chk_src_df.copy()
src_t_rev_day_df = src_t_rev_day_df.loc[src_t_rev_day_df['TM_KEY_DAY']>=v_tm_key_day]
src_t_rev_day_df = src_t_rev_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = src_t_rev_day_df.iloc[:, 4:11].columns.tolist()
for col in mod_col_list:
    src_t_rev_day_df[col] = src_t_rev_day_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\nCreate : src_t_rev_day_df...')
print(f'\n   -> DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
# print(f'\n{src_t_rev_day_df.to_string(max_cols=17)}')


Parameter input...

   -> v_tm_key_day: 20250101

Create : src_t_rev_day_df...

   -> DataFrame : 125 rows, 12 columns


In [26]:
''' TB1R000100 : Prepay Revenue '''

v_metric_cd = 'TB1R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB1R000100 : Prepay Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB1R000100_day_df = chk_src_df.copy()
TB1R000100_day_df = TB1R000100_day_df.loc[TB1R000100_day_df['METRIC_CD']==v_metric_cd]
TB1R000100_day_df = TB1R000100_day_df.loc[TB1R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB1R000100_day_df = TB1R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB1R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB1R000100_day_df[col] = TB1R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB1R000100_day_df


TB1R000100 : Prepay Revenue...

   -> v_metric_cd: TB1R000100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,jutar11,TB1R000100,Prepay Revenue,20250301,2025-03-11 04:46:38,0,0,"98,897,182","98,896,129","98,896,129","98,433,714",0
1,jutar11,TB1R000100,Prepay Revenue,20250302,2025-03-11 04:46:38,0,0,"97,911,164","97,910,912","97,910,912","97,449,429",0
2,jutar11,TB1R000100,Prepay Revenue,20250303,2025-03-11 04:46:38,0,0,"97,341,736","97,341,205","97,341,205","96,894,985",0
3,jutar11,TB1R000100,Prepay Revenue,20250304,2025-03-11 04:46:38,0,0,"87,058,292","87,057,790","87,057,790","86,686,172",0
4,jutar11,TB1R000100,Prepay Revenue,20250305,2025-03-11 04:46:38,0,0,"88,530,181","88,529,999","88,529,999","88,169,551",0
5,jutar11,TB1R000100,Prepay Revenue,20250306,2025-03-11 04:46:38,0,0,"82,875,276","82,875,070","82,875,070","82,538,757",0
6,jutar11,TB1R000100,Prepay Revenue,20250307,2025-03-11 04:46:38,0,0,"82,631,505","82,629,888","82,629,888","82,307,126",0
7,jutar11,TB1R000100,Prepay Revenue,20250308,2025-03-11 04:46:38,0,0,"78,439,101","78,438,838","78,438,838","78,113,977",0


In [27]:
''' TB2R010100 : Postpaid Revenue B2C '''

v_metric_cd = 'TB2R010100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB2R010100 : Postpaid Revenue B2C...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB2R010100_day_df = chk_src_df.copy()
TB2R010100_day_df = TB2R010100_day_df.loc[TB2R010100_day_df['METRIC_CD']==v_metric_cd]
TB2R010100_day_df = TB2R010100_day_df.loc[TB2R010100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB2R010100_day_df = TB2R010100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB2R010100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB2R010100_day_df[col] = TB2R010100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB2R010100_day_df


TB2R010100 : Postpaid Revenue B2C...

   -> v_metric_cd: TB2R010100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,jutar11,TB2R010100,Postpaid Revenue B2C,20250302,2025-03-11 04:20:53,0,"475,431,690","475,354,440","475,353,541","475,353,541","473,865,597",0
1,jutar11,TB2R010100,Postpaid Revenue B2C,20250306,2025-03-11 04:20:53,0,"279,692,047","279,664,064","279,664,064","279,664,064","278,360,515",0


In [28]:
''' TB3R000100 : Broadband Revenue '''

v_metric_cd = 'TB3R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB3R000100 : Broadband Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB3R000100_day_df = chk_src_df.copy()
TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['METRIC_CD']==v_metric_cd]
TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB3R000100_day_df = TB3R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB3R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB3R000100_day_df[col] = TB3R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB3R000100_day_df


TB3R000100 : Broadband Revenue...

   -> v_metric_cd: TB3R000100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,salisa,TB3R000100,Broadband Revenue,20250302,2025-03-11 05:17:44,0,0,"218,852,181","218,852,181","218,852,181","218,885,549",0
1,salisa,TB3R000100,Broadband Revenue,20250306,2025-03-11 05:17:44,0,0,"142,659,058","142,659,058","142,659,058","142,698,562",0


In [29]:
''' TB4R000100 : TVS Revenue '''

v_metric_cd = 'TB4R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB4R000100 : TVS Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB4R000100_day_df = chk_src_df.copy()
TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['METRIC_CD']==v_metric_cd]
TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB4R000100_day_df = TB4R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB4R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB4R000100_day_df[col] = TB4R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB4R000100_day_df


TB4R000100 : TVS Revenue...

   -> v_metric_cd: TB4R000100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
